In [15]:
import torch
from torch import nn
import torch.distributions as d
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
# from torch.utils.tensorboard import SummaryWriter
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [42]:
torch.manual_seed(0)
STATE_DIM = 5 #Including fake state CO2.
CUDA_ID = 1
dt = .2
T = 1000 #Run simulation for 1000 hours.
N = int(T / dt) 
T_span = np.linspace(0, T, N + 1)
T_span_tensor = torch.reshape(torch.Tensor(T_span), [1, N + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.

device = torch.device("".join(["cuda:",f'{CUDA_ID}']) if torch.cuda.is_available() else "cpu")
LR = 1e-3
niter = 20000
pretrain_iter = 1000
BATCH_SIZE = 1

In [43]:
obs_df_awb_eca_full = pd.read_csv('awb_eca_synthetic_sol_df.csv') #Must be link to raw Github output if in Colab.
obs_df_awb_eca = obs_df_awb_eca_full[obs_df_awb_eca_full['hour'] <= T] #Test with just first T hours of data.

In [44]:
obs_times = np.array(obs_df_awb_eca['hour'])
obs_means_awb_eca = torch.Tensor(np.array(obs_df_awb_eca.drop(columns = 'hour')))
obs_means_awb_eca_T = obs_means_awb_eca.T
obs_error_scale_awb_eca = torch.mean(obs_means_awb_eca_T, 1) * 0.1 #Observation noise set at 10% of respective observation means.

In [45]:
I_S_tensor = 0.001 + 0.0005 * torch.sin((2 * math.pi / (24 * 365)) * T_span_tensor) #Exogenous SOC input function
I_D_tensor = 0.0001 + 0.00005 * torch.sin((2 * math.pi / (24 * 365)) * T_span_tensor) #Exogenous DOC input function

In [46]:
temp_ref = 283

#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
s_SOC = 0.01
s_DOC = 0.01
s_MBC = 0.01
s_EEC = 0.01
s_CO2 = 0.01

sawb_eca_ss_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC, 's_CO2': s_CO2}

In [47]:
############################################################
##SOIL BIOGEOCHEMICAL MODEL TEMPERATURE RESPONSE FUNCTIONS##
############################################################

def temp_gen(t, temp_ref):
    temp = temp_ref + t / (20 * 24 * 365) + 10 * torch.sin((2 * np.pi / 24) * t) + 10 * torch.sin((2 * math.pi / (24 * 365)) * t)
    return temp

def arrhenius_temp_dep(parameter, temp, Ea, temp_ref):
    '''
    For a parameter with Arrhenius temperature dependence, returns the transformed parameter value.
    0.008314 is the gas constant. Temperatures are in K.
    '''
    decayed_parameter = parameter * torch.exp(-Ea / 0.008314 * (1 / temp - 1 / temp_ref))
    return decayed_parameter

def linear_temp_dep(parameter, temp, Q, temp_ref):
    '''
    For a parameter with linear temperature dependence, returns the transformed parameter value.
    Q is the slope of the temperature dependence and is a varying parameter.
    Temperatures are in K.
    '''
    modified_parameter = parameter - Q * (temp - temp_ref)
    return modified_parameter

##########################################################################
##DETERMINISTIC SOIL BIOGEOCHEMICAL MODEL INITIAL STEADY STATE ESTIMATES##
##########################################################################

def analytical_steady_state_init_awb_eca(SOC_input, DOC_input, sawb_eca_ss_params_dict = sawb_eca_ss_params_dict):
    '''
    Returns a vector of C pool values to initialize an SAWB-ECA system corresponding to set of parameter values using the analytical steady state equations of the deterministic CON system.
    Vector elements are in order of S_0, D_0, M_0, E_0, and CO2_0.
    Expected sawb_eca_ss_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC, 's_CO2': s_CO2}
    '''
    S_0 = ((-sawb_eca_ss_params_dict['K_DE'] * sawb_eca_ss_params_dict['r_L'] * (sawb_eca_ss_params_dict['r_E'] + sawb_eca_ss_params_dict['r_M']) * (sawb_eca_ss_params_dict['u_Q_ref'] - 1) + sawb_eca_ss_params_dict['r_E'] * sawb_eca_ss_params_dict['u_Q_ref'] * (SOC_input + DOC_input)) * (SOC_input * sawb_eca_ss_params_dict['r_E'] * (sawb_eca_ss_params_dict['u_Q_ref'] - 1) - sawb_eca_ss_params_dict['a_MSA'] * DOC_input * sawb_eca_ss_params_dict['r_M'] * sawb_eca_ss_params_dict['u_Q_ref'] + SOC_input * sawb_eca_ss_params_dict['r_M'] * (sawb_eca_ss_params_dict['u_Q_ref'] - sawb_eca_ss_params_dict['a_MSA'] * sawb_eca_ss_params_dict['u_Q_ref'] - 1))) / ((sawb_eca_ss_params_dict['r_E'] + sawb_eca_ss_params_dict['r_M']) * (sawb_eca_ss_params_dict['u_Q_ref'] - 1) * (DOC_input * sawb_eca_ss_params_dict['u_Q_ref'] * (sawb_eca_ss_params_dict['r_E'] * sawb_eca_ss_params_dict['V_DE_ref'] - sawb_eca_ss_params_dict['a_MSA'] * sawb_eca_ss_params_dict['r_L'] * sawb_eca_ss_params_dict['r_M']) + SOC_input * (sawb_eca_ss_params_dict['r_E'] * sawb_eca_ss_params_dict['r_L'] * (sawb_eca_ss_params_dict['u_Q_ref'] - 1) + sawb_eca_ss_params_dict['r_L'] * sawb_eca_ss_params_dict['r_M'] * (sawb_eca_ss_params_dict['u_Q_ref'] - sawb_eca_ss_params_dict['a_MSA'] * sawb_eca_ss_params_dict['u_Q_ref'] - 1) + sawb_eca_ss_params_dict['r_E'] * sawb_eca_ss_params_dict['u_Q_ref'] * sawb_eca_ss_params_dict['V_DE_ref'])))
    D_0 = -(sawb_eca_ss_params_dict['K_UE'] * (sawb_eca_ss_params_dict['r_E'] + sawb_eca_ss_params_dict['r_M']) * (sawb_eca_ss_params_dict['u_Q_ref'] - 1) - (SOC_input + DOC_input) * sawb_eca_ss_params_dict['u_Q_ref']) / ((sawb_eca_ss_params_dict['u_Q_ref'] - 1) * (sawb_eca_ss_params_dict['r_E'] + sawb_eca_ss_params_dict['r_M'] - sawb_eca_ss_params_dict['u_Q_ref'] * sawb_eca_ss_params_dict['V_UE_ref']))
    M_0 = -((SOC_input + DOC_input) * sawb_eca_ss_params_dict['u_Q_ref']) / ((sawb_eca_ss_params_dict['r_E'] + sawb_eca_ss_params_dict['r_M']) * (sawb_eca_ss_params_dict['u_Q_ref'] - 1))
    E_0 = sawb_eca_ss_params_dict['r_E'] * M_0 / sawb_eca_ss_params_dict['r_L']
    #E_0 = -((sawb_params_dict['r_E'] * sawb_params_dict['u_Q_ref'] * (SOC_input + DOC_input)) / (sawb_params_dict['r_L'] * (sawb_params_dict['r_E'] + sawb_params_dict['r_M']) * (sawb_params_dict['u_Q_ref'] - 1)))
    CO2_0 = (1 - sawb_eca_ss_params_dict['u_Q_ref']) * sawb_eca_ss_params_dict['V_UE_ref'] * M_0 * D_0 / (sawb_eca_ss_params_dict['K_UE'] + M_0 + D_0)
    C_0_vector = torch.as_tensor([S_0, D_0, M_0, E_0, CO2_0])
    return C_0_vector

In [48]:
####################################################
##STOCHASTIC DIFFERENTIAL EQUATION MODEL FUNCTIONS##
#################################################### 

def drift_diffusion_sawb_eca_ss(C_path, T_span_tensor, I_S_tensor, I_D_tensor, sawb_eca_ss_params_dict, temp_ref):
    '''
    Returns SAWB-ECA "state scaling diffusion parameterization" drift vectors and diffusion matrices.
    current_temp is output from temp_gen function. 
    Expected sawb_eca_ss_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC, 's_CO2': s_CO2}
    '''
    state_dim = 5 #SAWB and AWB family variants will have 5 'states' with the inclusion of CO2.
    SOC, DOC, MBC, EEC, CO2 =  torch.chunk(C_path, state_dim, -1) #Partition SOC, DOC, MBC, EEC, and CO2 values. Split based on final C_path dim, which specifies state variables and is also indexed as dim #2 in tensor. 
    current_temp = temp_gen(T_span_tensor, temp_ref) #Obtain temperature function vector across span of times.
    drift = torch.empty_like(C_path, device=C_path.device) #Initiate tensor with same dims as C_path to assign drift.
    diffusion_sqrt = torch.zeros([drift.size(0), drift.size(1), state_dim, state_dim], device = drift.device) #Create tensor to assign diffusion matrix elements. Diffusion exists for explicit algebraic variable CO2.
    #diffusion_sqrt_diag = torch.empty_like(C_path, device=C_path.device) #Create tensor to assign diffusion matrix elements.
    #Decay parameters are forced by temperature changes.
    u_Q = linear_temp_dep(sawb_eca_ss_params_dict['u_Q_ref'], current_temp, sawb_eca_ss_params_dict['Q'], temp_ref) #Apply linear temperature-dependence to u_Q.
    V_DE = arrhenius_temp_dep(sawb_eca_ss_params_dict['V_DE_ref'], current_temp, sawb_eca_ss_params_dict['Ea_V_DE'], temp_ref) #Apply vectorized temperature-dependent transformation to V_DE.
    V_UE = arrhenius_temp_dep(sawb_eca_ss_params_dict['V_UE_ref'], current_temp, sawb_eca_ss_params_dict['Ea_V_UE'], temp_ref) #Apply vectorized temperature-dependent transformation to V_UE.
    #Drift is calculated.
    drift_SOC = I_S_tensor + sawb_eca_ss_params_dict['a_MSA'] * sawb_eca_ss_params_dict['r_M'] * MBC - ((V_DE * EEC * SOC) / (sawb_eca_ss_params_dict['K_DE'] + EEC + SOC))
    drift_DOC = I_D_tensor + (1 - sawb_eca_ss_params_dict['a_MSA']) * sawb_eca_ss_params_dict['r_M'] * MBC + ((V_DE * EEC * SOC) / (sawb_eca_ss_params_dict['K_DE'] + EEC + SOC)) + sawb_eca_ss_params_dict['r_L'] * EEC - ((V_UE * MBC * DOC) / (sawb_eca_ss_params_dict['K_UE'] + MBC + DOC))
    drift_MBC = (u_Q * (V_UE * MBC * DOC) / (sawb_eca_ss_params_dict['K_UE'] + MBC + DOC)) - (sawb_eca_ss_params_dict['r_M'] + sawb_eca_ss_params_dict['r_E']) * MBC
    drift_EEC = sawb_eca_ss_params_dict['r_E'] * MBC - sawb_eca_ss_params_dict['r_L'] * EEC
    CO2 = (1 - u_Q) * (V_UE * MBC * DOC) / (sawb_eca_ss_params_dict['K_UE'] + MBC + DOC)
    #Assign elements to drift vector.
    drift[:, :, 0 : 1] = drift_SOC
    drift[:, :, 1 : 2] = drift_DOC
    drift[:, :, 2 : 3] = drift_MBC
    drift[:, :, 3 : 4] = drift_EEC
    drift[:, :, 4 : 5] = CO2 #CO2 is not a part of the drift. This is a hack for the explicit algebraic variable situation.
    #Diffusion matrix is assigned.
    diffusion_sqrt[:, :, 0 : 1, 0] = torch.sqrt(LowerBound.apply(SOC * sawb_eca_ss_params_dict['s_SOC'], 1e-9)) #SOC diffusion standard deviation
    diffusion_sqrt[:, :, 1 : 2, 1] = torch.sqrt(LowerBound.apply(DOC * sawb_eca_ss_params_dict['s_DOC'], 1e-9)) #DOC diffusion standard deviation
    diffusion_sqrt[:, :, 2 : 3, 2] = torch.sqrt(LowerBound.apply(MBC * sawb_eca_ss_params_dict['s_MBC'], 1e-9)) #MBC diffusion standard deviation
    diffusion_sqrt[:, :, 3 : 4, 3] = torch.sqrt(LowerBound.apply(EEC * sawb_eca_ss_params_dict['s_EEC'], 1e-9)) #EEC diffusion standard deviation
    diffusion_sqrt[:, :, 4 : 5, 4] = torch.sqrt(LowerBound.apply(CO2 * sawb_eca_ss_params_dict['s_CO2'], 1e-9)) #CO2 diffusion standard deviation
    #diffusion_sqrt_diag[:, :, 0 : 1] = torch.sqrt(LowerBound.apply(SOC * sawb_eca_ss_params_dict['s_SOC'], 1e-9)) #SOC diffusion standard deviation
    #diffusion_sqrt_diag[:, :, 1 : 2] = torch.sqrt(LowerBound.apply(DOC * sawb_eca_ss_params_dict['s_DOC'], 1e-9)) #DOC diffusion standard deviation
    #diffusion_sqrt_diag[:, :, 2 : 3] = torch.sqrt(LowerBound.apply(MBC * sawb_eca_ss_params_dict['s_MBC'], 1e-9)) #MBC diffusion standard deviation
    #diffusion_sqrt_diag[:, :, 3 : 4] = torch.sqrt(LowerBound.apply(EEC * sawb_eca_ss_params_dict['s_EEC'], 1e-9)) #EEC diffusion standard deviation
    #diffusion_sqrt_diag[:, :, 4 : 5] = torch.sqrt(LowerBound.apply(CO2 * sawb_eca_ss_params_dict['s_CO2'], 1e-9)) #CO2 diffusion standard deviation
    #diffusion_sqrt = torch.diag_embed(diffusion_sqrt_diag)
    return drift, diffusion_sqrt

In [49]:
class LowerBound(Function):
    @staticmethod
    def forward(ctx, inputs, bound):
        b = torch.ones(inputs.size()) * bound
        b = b.to(inputs.device)
        b = b.type(inputs.dtype)
        ctx.save_for_backward(inputs, b)
        return torch.max(inputs, b)
    @staticmethod
    def backward(ctx, grad_output):
        inputs, b = ctx.saved_tensors

        pass_through_1 = inputs >= b
        pass_through_2 = grad_output < 0

        pass_through = pass_through_1 | pass_through_2
        return pass_through.type(grad_output.dtype) * grad_output, None

class MaskedConv1d(nn.Conv1d):
    def __init__(self, mask_type, *args, **kwargs):
        super(MaskedConv1d, self).__init__(*args, **kwargs)
        assert mask_type in {'A', 'B'}
        self.register_buffer('mask', self.weight.data.clone())
        _, _, kW = self.weight.size()
        self.mask.fill_(1)
        self.mask[:, :, kW // 2 + 1*(mask_type == 'B'):] = 0

    def forward(self, x):
        self.weight.data *= self.mask
        return super(MaskedConv1d, self).forward(x)

class ResNetBlock(nn.Module):

    def __init__(self, inp_cha, out_cha, stride = 1, first = True, batch_norm = True):
        super().__init__()
        self.conv1 = MaskedConv1d('A' if first else 'B', inp_cha,  out_cha, 3, stride, 1, bias = False)
        self.conv2 = MaskedConv1d('B', out_cha,  out_cha, 3, 1, 1, bias = False)

        self.act1 = nn.PReLU(out_cha, init = 0.2)
        self.act2 = nn.PReLU(out_cha, init = 0.2)

        if batch_norm:
            self.bn1 = nn.BatchNorm1d(out_cha)
            self.bn2 = nn.BatchNorm1d(out_cha)
        else:
            self.bn1 = nn.Identity()
            self.bn2 = nn.Identity()

        # If dimensions change, transform shortcut with a conv layer
        if inp_cha != out_cha or stride > 1:
            self.conv_skip = MaskedConv1d('A' if first else 'B', inp_cha,  out_cha, 3, stride, 1, bias = False)
        else:
            self.conv_skip = nn.Identity()

    def forward(self, x):
        residual = x
        x = self.act1(self.bn1(self.conv1(x)))
        x = self.act2(self.bn2(self.conv2(x) + self.conv_skip(residual)))
        return x

class ResNetBlockUnMasked(nn.Module):
    
    def __init__(self, inp_cha, out_cha, stride = 1, batch_norm = False):
        super().__init__()
        self.conv1 = nn.Conv1d(inp_cha,  out_cha, 3, stride, 1)
        self.conv2 = nn.Conv1d(out_cha,  out_cha, 3, 1, 1)

        self.act1 = nn.PReLU(out_cha, init = 0.2)
        self.act2 = nn.PReLU(out_cha, init = 0.2)

        if batch_norm:
            self.bn1 = nn.BatchNorm1d(out_cha)
            self.bn2 = nn.BatchNorm1d(out_cha)
        else:
            self.bn1 = nn.Identity()
            self.bn2 = nn.Identity()

        # If dimensions change, transform shortcut with a conv layer
        if inp_cha != out_cha or stride > 1:
            self.conv_skip = nn.Conv1d(inp_cha,  out_cha, 3, stride, 1, bias=False)
        else:
            self.conv_skip = nn.Identity()

    def forward(self, x):
        residual = x
        x = self.act1(self.bn1(self.conv1(x)))
        x = self.act2(self.bn2(self.conv2(x) + self.conv_skip(residual)))
        return x

class CouplingLayer(nn.Module):

    def __init__(self, cond_inputs, stride, h_cha = 96):
        super().__init__()
        self.first_block = ResNetBlock(1, h_cha, first = True)
        self.second_block = nn.Sequential(ResNetBlock(h_cha + cond_inputs, h_cha, first = False),
                                          MaskedConv1d('B', h_cha,  2, 3, stride, 1, bias = False))

        self.feature_net = nn.Sequential(ResNetBlockUnMasked(cond_inputs, h_cha),
                                          ResNetBlockUnMasked(h_cha, cond_inputs))
        
        self.unpack = True if cond_inputs > 1 else False

    def forward(self, x, cond_inputs):
        if self.unpack:
            cond_inputs = torch.cat([*cond_inputs], 1)
        cond_inputs = self.feature_net(cond_inputs)
        first_block = self.first_block(x)
        feature_vec = torch.cat([first_block, cond_inputs], 1)
        output = self.second_block(feature_vec)
        mu, sigma = torch.chunk(output, 2, 1)
        sigma = LowerBound.apply(sigma, 1e-6)
        x = mu + sigma*x
        return x, -torch.log(sigma)

class PermutationLayer(nn.Module):

    def __init__(self):
        super().__init__()
        self.index_1 = torch.randperm(STATE_DIM)

    def forward(self, x):
        B, S, L = x.shape
        x_reshape = x.reshape(B, S, -1, STATE_DIM)
        x_perm = x_reshape[:, :, :, self.index_1]
        x = x_perm.reshape(B, S, L)
        return x

class SoftplusLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.softplus = nn.Softplus()
    
    def forward(self, x):
        y = self.softplus(x)
        return y, -torch.log(-torch.expm1(-y))

class BatchNormLayer(nn.Module):
    def __init__(self, num_inputs, momentum = 0.0, eps = 1e-5):
        super(BatchNormLayer, self).__init__()

        self.log_gamma = nn.Parameter(torch.rand(num_inputs))
        self.beta = nn.Parameter(torch.rand(num_inputs))
        self.momentum = momentum
        self.eps = eps

        self.register_buffer('running_mean', torch.zeros(num_inputs))
        self.register_buffer('running_var', torch.ones(num_inputs))

    def forward(self, inputs):
        inputs = inputs.squeeze(1)
        if self.training:
            self.batch_mean = inputs.mean(0)
            self.batch_var = (
                inputs - self.batch_mean).pow(2).mean(0) + self.eps

            self.running_mean.mul_(self.momentum)
            self.running_var.mul_(self.momentum)

            self.running_mean.add_(self.batch_mean.data *
                                   (1 - self.momentum))
            self.running_var.add_(self.batch_var.data *
                                  (1 - self.momentum))

            mean = self.batch_mean
            var = self.batch_var
        else:
            mean = self.running_mean
            var = self.running_var

        x_hat = (inputs - mean) / var.sqrt()
        y = torch.exp(self.log_gamma) * x_hat + self.beta
        ildj = -self.log_gamma + 0.5 * torch.log(var)
        return y[:, None, :], ildj[None, None, :]

In [50]:
class SDEFlow(nn.Module):

    def __init__(self, cond_inputs = 1, num_layers = 5):
        super().__init__()
        
        self.coupling = nn.ModuleList([CouplingLayer(cond_inputs + STATE_DIM, 1) for _ in range(num_layers)])
        self.permutation = [PermutationLayer() for _ in range(num_layers)]
        self.batch_norm = nn.ModuleList([BatchNormLayer(STATE_DIM * N) for _ in range(num_layers-1)])
        self.SP = SoftplusLayer()
        
        self.base_dist = d.normal.Normal(loc = 0., scale = 1.)
        self.num_layers = num_layers
        
    def forward(self, batch_size, obs_model, *args, **kwargs):

        eps = self.base_dist.sample([batch_size, 1, STATE_DIM * N]).to(device)
        log_prob = self.base_dist.log_prob(eps).sum(-1)
        
        obs_tile = obs_model.mu[None, :, 1:, None].repeat(batch_size, STATE_DIM, 1, 50).reshape(batch_size, STATE_DIM, -1)
        times = torch.arange(dt, T + dt, dt, device = eps.device)[(None,) * 2].repeat(batch_size, STATE_DIM, 1).transpose(-2, -1).reshape(batch_size, 1, -1)
        
        ildjs = []
        
        for i in range(self.num_layers):
            eps, cl_ildj = self.coupling[i](self.permutation[i](eps), (obs_tile, times))
            if i < (self.num_layers - 1):
                eps, bn_ildj = self.batch_norm[i](eps)
                ildjs.append(bn_ildj)
            ildjs.append(cl_ildj)
                
        eps, sp_ildj = self.SP(eps)
        ildjs.append(sp_ildj)
        
        for ildj in ildjs:
            log_prob += ildj.sum(-1)
    
        return eps.reshape(batch_size, STATE_DIM, -1).permute(0, 2, 1) + 1e-9, log_prob

In [51]:
class ObsModel(nn.Module):

    def __init__(self, times, mu, scale):
        super().__init__()

        self.idx = self._get_idx(times)
        self.times = times
        self.mu = torch.Tensor(mu).to(device)
        self.scale = scale
        
    def forward(self, x):
        obs_ll = d.normal.Normal(self.mu.permute(1, 0), self.scale).log_prob(x[:, self.idx, :])
        return torch.sum(obs_ll, [-1, -2]).mean()

    def _get_idx(self, times):
        return list((times / dt).astype(int))
    
    def plt_dat(self):
        return self.mu, self.times

In [52]:
def neg_log_lik(C_path, T_span_tensor, dt, I_S_tensor, I_D_tensor, drift_diffusion, params_dict, temp_ref):
    drift, diffusion_sqrt = drift_diffusion(C_path[:, :-1, :], T_span_tensor[:, :-1, :], I_S_tensor[:, :-1, :], I_D_tensor[:, :-1, :], params_dict, temp_ref)
    #print('\n drift =', drift)
    #print('\n diffusion_sqrt =', diffusion_sqrt)
    #euler_maruyama_sample = d.multivariate_normal.MultivariateNormal(loc = C_path[:, :-1, :] + drift * dt, scale_tril = diffusion_sqrt * math.sqrt(dt)) This line no longer applies because of addition of CO2 as a 'state'.
    drift_means_with_CO2 = torch.cat((C_path[:, :-1, :-1] + drift[:, :, :-1] * dt, drift[:, :, -1].unsqueeze(2)), 2) #Separate explicit algebraic variable CO2 mean from integration process.
    euler_maruyama_sample = d.multivariate_normal.MultivariateNormal(loc = drift_means_with_CO2, scale_tril = diffusion_sqrt * math.sqrt(dt))
    return -euler_maruyama_sample.log_prob(C_path[:, 1:, :]).sum(-1)

In [53]:
obs_model = ObsModel(times = obs_times, mu = obs_means_awb_eca_T, scale = obs_error_scale_awb_eca.reshape([1, STATE_DIM]))
net = SDEFlow().to(device)
optimizer = optim.Adam(net.parameters(), lr = LR)

def train(niter, pretrain_iter, BATCH_SIZE, T_span_tensor, I_S_tensor, I_D_tensor, drift_diffusion, params_dict, analytical_steady_state_init):
    if pretrain_iter >= niter:
        raise Exception("pretrain_inter must be < niter.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = analytical_steady_state_init(I_S_tensor[0, 0, 0].item(), I_D_tensor[0, 0, 0].item(), params_dict) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(device) #Assign initial conditions to C_path.
    with tqdm(total = niter, desc = f'Train Diffusion', position = -1) as t:
        for iter in range(niter):
            net.train()
            optimizer.zero_grad()
            C_path, log_prob = net(BATCH_SIZE, obs_model) #Obtain paths with solutions at times after t0.
            C_path = torch.cat([C0, C_path], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            if iter <= pretrain_iter:
                l1_norm_element = C_path - torch.mean(obs_model.mu, -1)
                l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                l1_norm.backward()
                norm_losses.append(l1_norm.item())
                #l2_norm_element = C_path - torch.mean(obs_model.mu, -1)
                #l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                #best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                #l2_norm.backward()
                #norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nC_path mean =', C_path.mean(-2))
                    print('\nC_path =', C_path)
            else:
                log_lik = neg_log_lik(C_path, T_span_tensor.to(device), dt, I_S_tensor.to(device), I_D_tensor.to(device), drift_diffusion, params_dict, temp_ref)
                ELBO = log_prob.mean() + log_lik.mean() - obs_model(C_path)
                best_loss_ELBO = ELBO if ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO.backward()
                ELBO_losses.append(ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nC_path mean =', C_path.mean(-2))
                    print('\n C_path =', C_path)
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            t.update()

In [ ]:
train(niter, pretrain_iter, BATCH_SIZE, T_span_tensor, I_S_tensor, I_D_tensor, drift_diffusion_sawb_eca_ss, sawb_eca_ss_params_dict, analytical_steady_state_init_awb_eca)


Train Diffusion:   0%|          | 1/20000 [00:03<21:24:51,  3.85s/it]

Moving average norm loss at 0 iterations is: 9000026948.89375. Best norm loss value is: 269488.9375.

C_path mean = tensor([[0.9220, 0.6072, 0.6381, 0.5925, 0.5082]], grad_fn=<MeanBackward1>)

C_path = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [7.9971e-01, 7.1143e-01, 6.3176e-01, 5.6106e-01, 5.3278e-01],
         [8.4864e-01, 6.5093e-01, 6.1483e-01, 6.3111e-01, 4.6989e-01],
         ...,
         [8.1543e-01, 6.3381e-01, 6.3075e-01, 4.7001e-01, 5.0232e-01],
         [9.2011e-01, 6.3572e-01, 6.5004e-01, 6.3174e-01, 5.7745e-01],
         [6.9366e-01, 6.7901e-01, 6.9061e-01, 6.2450e-01, 8.0403e-01]]],
       grad_fn=<CatBackward>)



Train Diffusion:   0%|          | 11/20000 [00:44<22:15:22,  4.01s/it]

Moving average norm loss at 10 iterations is: 256154.1609375. Best norm loss value is: 245591.890625.

C_path mean = tensor([[5.0919, 0.5965, 0.4443, 0.2204, 0.0907]], grad_fn=<MeanBackward1>)

C_path = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [1.3560e+00, 3.1710e+00, 5.3221e-01, 3.3265e-01, 1.4144e-01],
         [3.7443e+00, 2.1107e+00, 5.5668e-01, 3.1682e-01, 1.4090e-01],
         ...,
         [3.4383e+00, 5.4734e-01, 3.2404e-01, 1.4100e-01, 5.4285e-02],
         [4.7260e+00, 5.3687e-01, 3.2807e-01, 1.3682e-01, 6.4090e-02],
         [3.7674e+00, 5.3702e-01, 3.2240e-01, 1.3616e-01, 9.2043e-02]]],
       grad_fn=<CatBackward>)



Train Diffusion:   0%|          | 21/20000 [01:25<24:15:15,  4.37s/it]

...


Train Diffusion:   2%|▏         | 311/20000 [20:34<17:42:01,  3.24s/it]

Moving average norm loss at 310 iterations is: 2973.33447265625. Best norm loss value is: 2674.88623046875.

C_path mean = tensor([[5.3131e+01, 2.4302e-01, 7.3491e-01, 3.5438e-04, 3.7859e-07]],
       grad_fn=<MeanBackward1>)

C_path = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [1.4864e+01, 2.1966e+01, 5.4765e-01, 1.0000e-01, 2.4465e-07],
         [4.0514e+01, 7.9518e+00, 6.5195e-01, 5.6814e-03, 2.4622e-07],
         ...,
         [5.1774e+01, 2.8649e-01, 7.8418e-01, 1.0797e-06, 1.3087e-09],
         [5.1140e+01, 3.8176e-01, 6.8113e-01, 8.3433e-07, 1.6774e-09],
         [4.0095e+01, 4.6138e-01, 6.2250e-01, 3.7894e-07, 4.2048e-08]]],
       grad_fn=<CatBackward>)



Train Diffusion:   2%|▏         | 321/20000 [21:07<18:04:36,  3.31s/it]

Moving average norm loss at 320 iterations is: 2430.7921508789063. Best norm loss value is: 1995.1014404296875.

C_path mean = tensor([[5.3129e+01, 2.2076e-01, 7.2761e-01, 2.8919e-04, 4.0987e-07]],
       grad_fn=<MeanBackward1>)

C_path = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [1.5559e+01, 2.1467e+01, 5.2392e-01, 1.1259e-01, 2.2170e-07],
         [4.1695e+01, 7.5248e+00, 6.3514e-01, 5.0993e-03, 2.1598e-07],
         ...,
         [5.2018e+01, 2.2838e-01, 7.9066e-01, 1.0901e-06, 1.4303e-09],
         [5.0703e+01, 3.1822e-01, 6.9303e-01, 8.0555e-07, 1.7105e-09],
         [3.9550e+01, 4.1800e-01, 6.7990e-01, 3.5554e-07, 3.4804e-08]]],
       grad_fn=<CatBackward>)



Train Diffusion:   2%|▏         | 322/20000 [21:11<18:59:37,  3.47s/it]